# Task 3

In [2]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import pymongo

from pymongo import MongoClient

import json

from pprint import pprint

import sys

"""connect to monoDB"""

client = MongoClient()

db = client.new2

## NER filter 

In [17]:
total4 = db.NERfilter
tweets = db.total3.find()
for tweet in tweets:
    keys = tweet.keys()
    doc = nlp(tweet['text'])
    labels = [x.label_ for x in doc.ents]
    Counter(labels)
    if ('PERSON' in labels or 'LOC' in labels or 'ORG' in labels):
        total4.insert_one(tweet)
    else:
        pass

## NER filter 2 PERSON/LOC/ORG labels 

In [2]:
total5 = db.noise_filter
total4 = db.NERfilter

sets = total4.find()

nlp = en_core_web_sm.load()

for set in sets:
    doc = nlp(set['text'])
    orgstr = ""
    locstr = ""
    pstr = ""
    for X in doc.ents:
        str = X.label_
        if (str is "ORG"):
            orgstr = orgstr + "," + X.text
        elif (str is "LOC"):
            locstr = locstr + "," + X.text
        elif (str is "PERSON"):
            pstr = pstr + "," + X.text

    if (orgstr is not "") or (locstr is not "") or (pstr is not ""):
        id=set['id']

        total5.insert_one(set)

        myquery = {"id": id}
        newvalues = {"$set": {"ORG": orgstr,"LOC": locstr,"PERSON": pstr}}
        total5.update_one(myquery, newvalues)

## extract PERSON/ORG/LOC labels from text  

In [5]:
total5 = db.noise_filter
total6 = db.addtag

sets = total5.find()

nlp = en_core_web_sm.load()

for set in sets:
    doc = nlp(set['text'])

    org = 0
    loc = 0
    person = 0

    orgstr = ""
    locstr = ""
    pstr = ""


    for X in doc.ents:
        str = X.label_
        if ord(X.text[0])>64 and ord(X.text[0])<123:
            if (str is "ORG"):
                if(i1==0):
                    orgstr = X.text.lower()
                    org==1

            elif (str is "LOC"):
                if(i2==0):
                    locstr =  X.text.lower()
                    loc=1

            elif (str is "PERSON"):
                if(i3==0):
                    pstr = X.text.lower()
                    person=1





    if (orgstr is not "") or (locstr is not "") or (pstr is not ""):
        id=set['id']

        total6.insert_one({"id":id,"ORG": orgstr,"LOC": locstr,"PERSON": pstr,"total_tag": 0,"org_tag":0,"person_tag":0,"loc_tag":0,"Amount":0})


##  add tags for PERSON/LOC/ORG labels 

In [12]:
total6 = db.addtag

import operator


DISTANCE = 10000


orgtagged = []
persontagged = []
loctagged = []


DATASET= total6.aggregate([{"$group":{"_id":"$PERSON"}}])
for data in DATASET:
    if not operator.eq(data['_id'],""):
        persontagged.append(data['_id'])
print(len(persontagged))

DATASET= total6.aggregate([{"$group":{"_id":"$ORG"}}])
for data in DATASET:
    if not operator.eq(data['_id'], ""):
        orgtagged.append(data['_id'])
print(len(orgtagged))

DATASET= total6.aggregate([{"$group":{"_id":"$LOC"}}])
for data in DATASET:
    if not operator.eq(data['_id'], ""):
        loctagged.append(data['_id'])
print(len(loctagged))


#step 1

tag = 0
tag1 = DISTANCE
tag2 = DISTANCE * 2
tag3 = DISTANCE * 3

orgONE = []
personONE = []
locONE = []

query = {"total_tag": {'$ne':0}}
addvalues = {"$set": {"total_tag": 0,"org_tag":0,"person_tag":0,"loc_tag":0}}
total6.update_many(query, addvalues)

for i in range(len(orgtagged)):
    ones = total6.find()
    for one in ones:
        if (operator.eq(one['ORG'],orgtagged[i])):
            orgONE.append(orgtagged[i])
            query = {"_id": one['_id']}
            t=one['total_tag']+1
            addvalues = {"$set": {"NG": tag,"org_tag":1,"total_tag":t}}
            total6.update_many(query, addvalues)
    tag = tag + 1
print("org",tag)

for i in range(len(persontagged)):
    ones = total6.find()
    for one in ones:
        if (operator.eq(one['PERSON'],persontagged[i])):
            personONE.append(persontagged[i])
            query = {"_id": one['_id']}
            t = one['total_tag'] + 1
            newvalues = {"$set": {"NG": tag1,"person_tag":1,"total_tag":t}}
            total6.update_many(query, addvalues)
    tag1 = tag1 + 1
print("person",tag1)

for i in range(len(loctagged)):
    ones = total6.find()
    for one in ones:
        if (operator.eq(one['LOC'],loctagged[i])):
            locONE.append(loctagged[i])
            query = {"_id": one['_id']}
            t = one['total_tag'] + 1
            addvalues = {"$set": {"NG": tag2,"loc_tag":1,"total_tag":t}}
            total6.update_many(query, addvalues)
    tag2 = tag2 + 1
print("loc",tag2)

DATASET= total6.aggregate([{"$group":{"_id":"$NG"}}])

i=0
for data in DATASET:
    i+=1

ones = total6.find()
for one in ones:
    if (one['total_tag']==3):
        query =  {'$or':[{"LOC": one['LOC']},{"PERSON": one['PERSON']},{"ORG": one['ORG']}]}
        addvalues = {"$set": {"NG": tag3,"total_tag":4}}
        total6.update_many(query, addvalues)
        tag3 += 1


    elif(one['total_tag']==2):

        if one['org_tag']==1 and one['loc_tag']==1:
            query = {'$or': [{"LOC": one['LOC']},{"ORG": one['ORG']}]}
            addvalues = {"$set": {"NG": tag3, "total_tag": 5}}
        elif one['org_tag']==1 and one['person_tag']==1:
            query = {'$or': [{"PERSON": one['PERSON']},{"ORG": one['ORG']}]}
            addvalues = {"$set": {"NG": tag3, "total_tag": 5}}
        elif one['person_tag']==1 and one['loc_tag']==1:
            query = {'$or': [{"LOC": one['LOC']},{"PERSON": one['PERSON']}]}
            addvalues = {"$set": {"NG": tag3, "total_tag": 5}}

        total6.update_many(query, addvalues)
        tag3 += 1

DATASET= total6.aggregate([{"$group":{"_id":"$NG"}}])

i=0
for data in DATASET:
    i+=1
print(i)


2442
2832
118
org 2832
person 12442
loc 20118
4434


## update the amount of each tag 

In [12]:
DATASET= total6.aggregate([{"$group":{"_id":"$NG"}}])

i=0
for data in DATASET:
    ngs = total6.find({'NG': data['_id']})
    count = 0
    for ng in ngs:
        count+=1

    query = {"NG":data['_id'] }
    addvalues = {"$set": {"Amount": count}}
    total6.update_many(query, addvalues)
    i+=1
print(i)

## create a new database to store grouped data 

In [4]:
total7 = db.grouping
total6 = db.addtag

query = {"Amount": {'$ne':1}}

DATASET= total6.find(query)

for data in DATASET:
    total7.insert_one(data)

## print the amount of groups  

In [5]:
DATASET= total7.aggregate([{"$group":{"_id":"$NG"}}])

i=0
for data in DATASET:
    i+=1
print(i)

564
